## **MIDI Factory**

#### Import Libraries

In [112]:
import mido
import pandas as pd
import xlwings as xw



#### Create empty Series

In [113]:
empty_series = pd.Series()

### Create DataFrame with empty Series

In [114]:
midi_df = pd.DataFrame({
    'note': empty_series,
    'note_start_time': empty_series,
    'duration': empty_series,
    'Velocity': empty_series,
    'heard_interval': empty_series,
    'theme_begin':empty_series,
    'theme_end':empty_series,
    
})

### Load your MIDI file

In [115]:
mid = mido.MidiFile('2000.mid')

In [123]:
# Initialize a dictionary to store note start times and velocities
note_start_times = {}
note_velocities = {}
techniques = {}
bar_duration = 0

for i, track in enumerate(mid.tracks):
    
    total_time = 0  # Initialize total time for each track
    end_time = 0    # Initialize end time for rests
    last_note_end_time = 0
    quarter_tone = False
    
    for message in track:
        total_time += message.time  # Update total time for each message
        
        if message.type == "marker":
            marker_time = message.time
            marker_titles = message.text.split(",")
            
            for marker_title in marker_titles:
            
                if marker_time in techniques:
                    techniques[marker_time].append(marker_title)
                else:
                    techniques[marker_time] = [marker_title]
                
                if marker_title not in midi_df.columns:
                    if midi_df.shape[0]:
                        midi_df[marker_title] = 0
                    else:
                        midi_df[marker_title] = empty_series
        
        elif message.type == 'pitchwheel':
            if (message.pitch):
                quarter_tone = not quarter_tone
        
        elif message.type == 'note_on' and message.velocity > 0:
            # Store the start time and velocity of the note
            note_start_times[message.note] = total_time
            note_velocities[message.note] = message.velocity
            
        elif message.type == 'note_off':
            # Calculate the duration using the stored start time
            start_time = note_start_times.get(message.note, 0)
            duration = total_time - start_time
            velocity = note_velocities.get(message.note, 0)
            
            # If there is rest before note
            if (start_time - last_note_end_time > 0):
                rest_start_time = last_note_end_time
                rest_total_duration = start_time - last_note_end_time
                
                rest_starts_bar = rest_start_time // bar_duration
                rest_ends_bar = (start_time - 1) // bar_duration    
                
                if(rest_starts_bar != rest_ends_bar):
                    rest_row_temp = {key: 0 for key in midi_df.columns}
                    for i in range(rest_ends_bar - rest_starts_bar):
                        bar_ends = (rest_starts_bar + 1 + i) * bar_duration
                        rest_row_temp = {key: 0 for key in midi_df.columns}
                        rest_row_temp['note'] = 0
                        rest_row_temp['note_start_time'] = rest_start_time
                        rest_row_temp['duration'] = bar_ends - rest_start_time
                        
                        rest_markers = []
                        if ( techniques.get(rest_start_time) ):
                            rest_markers = techniques.get(rest_start_time)
                        
                        if(rest_markers and len(rest_markers)):
                            for rest_marker in rest_markers:
                                rest_row_temp[rest_marker] = 1
                        
                        midi_df = pd.concat([midi_df, pd.DataFrame(rest_row_temp, index=[0])], ignore_index=True)
                        
                        rest_start_time = bar_ends
                        last_note_end_time = bar_ends
                        
                    if(start_time - rest_start_time):
                        last_rest_duration = start_time - rest_start_time                    
                        
                        rest_row_temp = {key: 0 for key in midi_df.columns}
                        rest_row_temp['note'] = 0
                        rest_row_temp['note_start_time'] = rest_start_time
                        rest_row_temp['duration'] = last_rest_duration
                        midi_df = pd.concat([midi_df, pd.DataFrame(rest_row_temp, index=[0])], ignore_index=True)
                        last_note_end_time = bar_ends
                else: 
                    rest_row_temp = {key: 0 for key in midi_df.columns}
                    rest_row_temp['note'] = 0
                    rest_row_temp['note_start_time'] = rest_start_time
                    rest_row_temp['duration'] = rest_total_duration
                    
                    rest_markers = []
                    if ( techniques.get(rest_start_time) ):
                        rest_markers = techniques.get(rest_start_time)
                    
                    if(rest_markers and len(rest_markers)):
                        for rest_marker in rest_markers:
                            rest_row_temp[rest_marker] = 1
                        
                    midi_df = pd.concat([midi_df, pd.DataFrame(rest_row_temp, index=[0])], ignore_index=True)
                    last_note_end_time = bar_ends
                    # print(f'Rest, Start Time: {last_note_end_time}, Duration: {start_time - last_note_end_time}')
            
            row_temp = {key: 0 for key in midi_df.columns}
            
            if quarter_tone:
                row_temp['note'] = message.note - 0.5
                quarter_tone = not quarter_tone
            else:
                row_temp['note'] = message.note
            
            row_temp['note_start_time'] = start_time
            row_temp['duration'] = duration
            row_temp['Velocity'] = velocity
            
            markers = []
            if ( techniques.get(start_time) ):
                markers = techniques.get(start_time)
            
            if(markers and len(markers)):
                for marker in markers:
                    row_temp[marker] = 1
            # print(f'Note: {message.note}, Start_Time: {start_time}, Duration: {duration}, Velocity: {velocity}')
            
            # Convert the dictionary to a DataFrame and append it to the original DataFrame
            midi_df = pd.concat([midi_df, pd.DataFrame(row_temp, index=[0])], ignore_index=True)
            
            last_note_end_time = start_time + duration
        elif (message.type == "time_signature"):
            bar_duration = message.numerator * message.denominator * message.clocks_per_click
            
# Connect to the workbook
wb = xw.Book('test.xlsx')

# Select the worksheet
ws = wb.sheets['Sheet1']

# Delete the existing table (if it exists)
if ws.tables:
    ws.range(ws.tables[0].range.address).api.Delete()

# Add a new table to the worksheet
table = ws.tables.add(source=ws.range('A1'), name='MyTable')

# Update the table with the new data
table.update(midi_df)

# Show the updated range of cells in a new window
table.range.api.Select()

True

In [118]:
# midi_df

In [119]:
# midi_df.to_excel("test.xlsx")

In [120]:
400 % 384

16

In [121]:
700 // 384

1